READ HTML TO CSV

In [63]:
import glob
import pandas as pd
import os
import numpy as np

In [64]:
def read_name_file(link,format):
    txt_files = glob.glob(os.path.join(link,'*.{0}'.format(format)))
    return txt_files

In [65]:
def webscraping(link):
    file_html = link   
    with open(file_html,mode = 'r',encoding='utf-8-sig' ) as file:
        html = file.read()
    title = []
    row = []
    column = []
    value = []
    for i in html.split("\n"):
        if i.startswith('<TITLE>'):
            title.append(i.split('>')[1].split('<')[0])
        if i.startswith('<TH VALIGN COLSPAN=1>'):
            column.append(i.split('>')[1].split('<')[0])
        if i.startswith('<TH ALIGN=LEFT VALIGN=TOP>'):
            row.append(i.split('>')[1].split('<')[0])
        if i.startswith('<TD>'):
            value.append(i.split('>')[1].split('<')[0])   
    return title,row,column,value


In [66]:
def to_df(row,column,value):
    datas = []
    data = []
    df = pd.DataFrame(columns=column, index=row)
    for i in value:
        data.append(i)
        if len(data) == len(column):
            datas.append(data)
            data = []
    for i, row_data in enumerate(datas):
        df.loc[row[i]] = row_data
    return df

In [67]:
def save_to_csv(df,title,link):
    df.to_csv('{0}{1}.csv'.format(link,str(title)),encoding='utf-8-sig',index=True) 
    return

In [68]:
a = read_name_file('E:/Learn_python/Project ADY/','html')
for i in a:
    title,row,column,value = webscraping(i)
    for i in range(len(value)):
        value[i] = value[i].replace('.','').replace(',','.')           
    df = to_df(row,column,value)
    title = str(title[0]).replace("*",'')
    print(df) 
    save_to_csv(df,title,f'E:/Learn_python/Project ADY/file csv/')

                2010    2011    2012    2013    2014   2015   2016    2017  \
CHỈ SỐ CHUNG  112.63  118.43  103.43  105.25  103.26  99.42  99.39  102.82   

                2018    2019   2020    2021    2022  
CHỈ SỐ CHUNG  103.09  101.25  99.40  102.91  104.24  
                2012    2013    2014    2015    2016    2017    2018    2019  \
CHỈ SỐ CHUNG  103.53  103.05  102.98  101.28  102.38  103.63  103.00  103.06   

               2020    2021    2022  
CHỈ SỐ CHUNG  99.27  100.95  103.73  
         2010    2011   2012    2013    2014   2015    2016   2017    2018  \
Ngành  114.40  131.60  97.87  100.57  104.62  99.72  101.71  99.92  101.98   

         2019    2020    2021    2022  
Ngành  101.31  108.79  102.12  103.89  
                  2010    2011    2012    2013    2014    2015    2016  \
Năm trước =100  109.19  118.58  109.21  106.60  104.09  100.63  102.66   

                  2017    2018    2019    2020    2021    2022  
Năm trước =100  103.53  103.54  102.79  103.23 

TO DATABASE

In [69]:
import mysql.connector

host = "127.0.0.1"
user = "root"
password = "khanhuyen1410"


conn = mysql.connector.connect(
    host=host,
    user=user,
    password=password,
)

if conn.is_connected():
    print("Kết nối đến MySQL thành công!")

cur = conn.cursor()


Kết nối đến MySQL thành công!


In [70]:
cur.execute("CREATE DATABASE ADYprojet3")
cur.execute("USE ADYprojet3")

In [71]:
def read_csv(link):
    df = pd.read_csv(link)
    return df

In [72]:
def create_table(name_table,name_column):
    list_column = []
    list_column.append(str(name_column[0]).replace(': 0',''))
    column = str(name_column[0]).replace(': 0','') + ' NVARCHAR(255),'

    for i in name_column[1:]:
        try:
            b = int(i)
            b = 'year_' + str(i)
            list_column.append(b)
            c = (b + ' FLOAT,')
            
        except:
            a = str(i).replace('(',' ').replace(')',' ').replace(',','_').replace(' ','_')
            list_column.append(a)
            c = (a + ' FLOAT,')
        column += c
    table = """CREATE TABLE {0}({1});""".format(name_table,column)
    return table.replace(',);',');'),list_column

In [73]:
def insert_value(name_table,df,name_column,list_column):
    values = []
    data_insert = []
    list_col = ''
    for i in list_column:
        list_col += str(i)+','
    list_col = list(list_col)
    list_col[-1] = ''
    list_col = ''.join(list_col)
    
    for i in name_column:
        values.append(list(df[i]))
    for i in range(len(values[0])):
        values[0][i] = "N'{0}'".format(values[0][i])
    print(values)
    data_np = np.array(values)
    transpose = np.transpose(data_np)
    for i in transpose:
        a = '''INSERT INTO {0} ({1})  VALUES ('''.format(name_table,list_col)
        for j in i:
            a += (str(j).replace(',','.') + ',')
        data_insert.append((a+');').replace(',);',');'))
    return data_insert
        

In [74]:
a = read_name_file('E:/Learn_python/Project ADY/file csv','csv')
count = 0
for i in a:
    df = read_csv(i)
    num_col= df.shape[1]
    name_column = list(df.iloc[0:0,0:(num_col)])
    table= create_table('a' + str(count),name_column)
    cur.execute(table[0])
    values = insert_value('a' +str(count),df,name_column,table[1])
    print(value)
    for i in values:
        cur.execute(i)
    count += 1

[["N'CHỈ SỐ CHUNG'"], [112.63], [118.43], [103.43], [105.25], [103.26], [99.42], [99.39], [102.82], [103.09], [101.25], [99.4], [102.91], [104.24]]
['11000.30', '11000.10', '10046.60', '11500.00', '12500.00', '14500.00', '15800.00', '17500.00', '19100.00', '20380.00', '19980.00', '19740.00', '22396.00']
[["N'CHỈ SỐ CHUNG'"], [103.53], [103.05], [102.98], [101.28], [102.38], [103.63], [103.0], [103.06], [99.27], [100.95], [103.73]]
['11000.30', '11000.10', '10046.60', '11500.00', '12500.00', '14500.00', '15800.00', '17500.00', '19100.00', '20380.00', '19980.00', '19740.00', '22396.00']
[["N'Ngành'"], [114.4], [131.6], [97.87], [100.57], [104.62], [99.72], [101.71], [99.92], [101.98], [101.31], [108.79], [102.12], [103.89]]
['11000.30', '11000.10', '10046.60', '11500.00', '12500.00', '14500.00', '15800.00', '17500.00', '19100.00', '20380.00', '19980.00', '19740.00', '22396.00']
[["N'Năm trước =100'"], [109.19], [118.58], [109.21], [106.6], [104.09], [100.63], [102.66], [103.53], [103.54]